In [3]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [12]:
df = pd.read_csv("homeprices_banglore.csv")
df.head(5)

,area,bedrooms,price
0,1056,2,39.07
1,2600,4,120.00
2,1440,3,62.00
3,1521,3,75.00
4,1200,2,51.00


In [14]:
from sklearn import preprocessing

sx = preprocessing.MinMaxScaler()
sy = preprocessing.MinMaxScaler()

scaled_x = sx.fit_transform(df.drop('price',axis='columns'))
scaled_x

array([[0.08827586, 0.25      ],
       [0.62068966, 0.75      ],
       [0.22068966, 0.5       ],
       [0.24862069, 0.5       ],
       [0.13793103, 0.25      ],
       [0.12758621, 0.25      ],
       [0.6662069 , 0.75      ],
       [0.86206897, 0.75      ],
       [0.17586207, 0.5       ],
       [1.        , 1.        ],
       [0.34482759, 0.5       ],
       [0.68448276, 0.75      ],
       [0.06896552, 0.25      ],
       [0.10344828, 0.25      ],
       [0.5       , 0.5       ],
       [0.12931034, 0.25      ],
       [0.13103448, 0.5       ],
       [0.25517241, 0.5       ],
       [0.67931034, 0.5       ],
       [0.        , 0.        ]])

In [17]:
### some examples of small fn's
### np.ones returns ones with given shape
a = np.ones(5,dtype=int) 
print('single arr with 5 ones',a)
b = np.ones([4,2], dtype=int)
print('4 by 2 arr with ones', b)
b.shape[0]

np.ones(shape=(2))

single arr with 5 ones [1 1 1 1 1]
4 by 2 arr with ones [[1 1]
 [1 1]
 [1 1]
 [1 1]]


array([1., 1.])

In [ ]:
def batch_gradient_descent( x , y_true , epochs , learning_rate = 0.1 ):
    number_of_features = x.shape[1] ## gives no.of.columns (ie . 2)
    w = np.ones(shape=(number_of_features)) ### w = [1,1] 
    b = 0
    total_samples = x.shape[0]

    for i in range(epochs):
        y_predicted = np.dot(w, scaled_x.T) + bias  ### exactly our ML model - pls refer the note - w1 * area + w2 * bedrooms

        ### since we know 
        ### w1 = w1 - learning rate * d/dw1
        ### w2 = w2 - learning rate * d/dw2
        ### bias = bias - learning rate * d/dw2

        ## so , firstly we need to find the derivatives part

        weight_derivative = -(2/total_samples)*(x.T.dot(y_true-y_predicted))
        bias_derivative = -(2/total_samples)*np.sum(y_true-y_predicted)

        weight = weight - learning_rate * weight_derivative
        bias = bias - learning_rate * bias_derivative

        cost = np.mean(np.square(y_true-y_predicted))

        if i % 10==0:
            cost_list.append(cost)
            epoch_list.append(i)
        